
### Rosenbrock Minimization with Backtracking Line Search

In [ ]:

# 1) Imports
import numpy as np
import pandas as pd

# Backtracking parameters (Armijo)
ALPHA_BAR = 1.0
RHO       = 0.5
C_ARMIJO  = 1e-4

# Stopping
TOL       = 1e-8
MAX_SD_IT = 2000      # for steepest descent
MAX_NT_IT = 200       # for Newton


In [ ]:
# 2) Rosenbrock function, gradient, Hessian
def rosenbrock(x):
    x1, x2 = x[0], x[1]
    return 100.0*(x2 - x1**2)**2 + (1.0 - x1)**2

def grad_rosenbrock(x):
    x1, x2 = x[0], x[1]
    df_dx1 = -400.0*x1*(x2 - x1**2) - 2.0*(1.0 - x1)
    df_dx2 = 200.0*(x2 - x1**2)
    return np.array([df_dx1, df_dx2])

def hess_rosenbrock(x):
    x1, x2 = x[0], x[1]
    h11 = 1200.0*x1**2 - 400.0*x2 + 2.0
    h12 = -400.0*x1
    h22 = 200.0
    return np.array([[h11, h12],[h12, h22]])


In [ ]:
# 3) Backtracking line search (Armijo)
def backtracking(f, grad, xk, pk, alpha_bar=ALPHA_BAR, rho=RHO, c=C_ARMIJO, max_backtracks=60):
    alpha = alpha_bar
    fk = f(xk)
    gk = grad(xk)
    slope0 = np.dot(gk, pk)
    # Ensure descent (rarely needed for Newton if Hessian is PD)
    if slope0 >= 0:
        pk = -pk
        slope0 = np.dot(gk, pk)
    bt = 0
    # Armijo condition
    while f(xk + alpha*pk) > fk + c*alpha*slope0 and bt < max_backtracks:
        alpha *= rho
        bt += 1
    return alpha, pk, bt


In [ ]:
# 4) Steepest Descent and Newton's Method
def steepest_descent(x0, max_iter=MAX_SD_IT, tol=TOL):
    x = np.array(x0, dtype=float)
    records = []
    for k in range(max_iter):
        g = grad_rosenbrock(x)
        gn = np.linalg.norm(g)
        fval = rosenbrock(x)
        if gn <= tol:
            records.append((k, x[0], x[1], fval, gn, np.nan, 0))
            break
        pk = -g
        alpha, pk_adj, bt = backtracking(rosenbrock, grad_rosenbrock, x, pk)
        x = x + alpha*pk_adj
        records.append((k, x[0], x[1], rosenbrock(x), np.linalg.norm(grad_rosenbrock(x)), alpha, bt))
    df = pd.DataFrame(records, columns=["iter","x1","x2","f(x)","||grad||","alpha","backtracks"])
    return df

def newton_method(x0, max_iter=MAX_NT_IT, tol=TOL):
    x = np.array(x0, dtype=float)
    records = []
    for k in range(max_iter):
        g = grad_rosenbrock(x)
        gn = np.linalg.norm(g)
        fval = rosenbrock(x)
        if gn <= tol:
            records.append((k, x[0], x[1], fval, gn, np.nan, 0))
            break
        H = hess_rosenbrock(x)
        # Solve H p = -g; fall back to least squares if singular/ill-conditioned
        try:
            pk = -np.linalg.solve(H, g)
        except np.linalg.LinAlgError:
            pk, *_ = np.linalg.lstsq(H, -g, rcond=None)
        alpha, pk_adj, bt = backtracking(rosenbrock, grad_rosenbrock, x, pk)
        x = x + alpha*pk_adj
        records.append((k, x[0], x[1], rosenbrock(x), np.linalg.norm(grad_rosenbrock(x)), alpha, bt))
    df = pd.DataFrame(records, columns=["iter","x1","x2","f(x)","||grad||","alpha","backtracks"])
    return df


In [ ]:
# 5) Runs and results
x0_easy = (1.2, 1.2)
x0_hard = (-1.2, 1.0)

sd_easy = steepest_descent(x0_easy)
sd_hard = steepest_descent(x0_hard)

nt_easy = newton_method(x0_easy)
nt_hard = newton_method(x0_hard)

# Display tables
sd_easy, sd_hard, nt_easy, nt_hard


(      iter        x1        x2      f(x)   ||grad||     alpha  backtracks
 0        0  1.087109  1.246875  0.430975  30.985565  0.000977          10
 1        1  1.114571  1.234166  0.019689   4.168659  0.000977          10
 2        2  1.110820  1.235749  0.012615   0.694695  0.000977          10
 3        3  1.111397  1.235392  0.012413   0.143725  0.000977          10
 4        4  1.111126  1.235318  0.012400   0.172884  0.001953           9
 ...    ...       ...       ...       ...        ...       ...         ...
 1995  1995  1.005816  1.011669  0.000034   0.010117  0.001953           9
 1996  1996  1.005796  1.011668  0.000034   0.009924  0.001953           9
 1997  1997  1.005806  1.011651  0.000034   0.009738  0.001953           9
 1998  1998  1.005788  1.011650  0.000034   0.009557  0.001953           9
 1999  1999  1.005797  1.011634  0.000034   0.009382  0.001953           9
 
 [2000 rows x 7 columns],
       iter        x1        x2      f(x)   ||grad||     alpha  backtrac

In [ ]:
# Save CSVs
sd_easy.to_csv("SD_x0_1p2_1p2.csv", index=False)
sd_hard.to_csv("SD_x0_-1p2_1.csv", index=False)
nt_easy.to_csv("NM_x0_1p2_1p2.csv", index=False)
nt_hard.to_csv("NM_x0_-1p2_1.csv", index=False)

print("Saved:", "SD_x0_1p2_1p2.csv", "SD_x0_-1p2_1.csv", "NM_x0_1p2_1p2.csv", "NM_x0_-1p2_1.csv")


Saved: SD_x0_1p2_1p2.csv SD_x0_-1p2_1.csv NM_x0_1p2_1p2.csv NM_x0_-1p2_1.csv
